<br><br>
# <strong>RNN
<strong>Recurrent Neual Network : 순환신경망 (순서가 있는 데이터를 처리하기에 용이하다)</strong>
1. <strong>tf.reset_default_graph()</strong> : tf 매개변수를 초기화 cf)Session.close() 는 모델만 종료, 매개변수는 계속 상존해 재사용시 충돌한다

https://github.com/tensorflow/tensorflow/issues/799


<br>
## <strong>1 RNN 분석 
<strong>by MNIST
1. X1, X2... X7 : 각 7개 매개변수가 단계별 구분된 데이터를 입력한다
1. S : Cell이란 중심으로써 심층 신경망을 생성한다

<img src="https://raw.githubusercontent.com/peterroelants/peterroelants.github.io/master/notebooks/RNN_implementation/img/SimpleRNN02_1.png" align="left" width='400'>

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

/home/markbaum/Python/python36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


### <strong>01 데이터 입력 및 모델의 파라미터 정의
Data

In [2]:
# 1. 입력값 및 파라미터를 정의
learning_rate = 0.001
total_epoch = 30
batch_size = 128

# RNN 은 순서가 있는 (ex)시계열) 자료를 다루는 모델이다
n_input = 28  # n_input (W_hh) : 가로 픽셀수로 한 번에 입력받는 갯수 
n_step  = 28  # n_step  (W_hy) : 세로 픽셀수로 총 몇 단계 데이터를 받을지를 설정
n_hidden, n_class = 128, 10

### <strong>02 RNN 모델 및 손실함수의 정의
1. 모델에 사용할 <strong>매개변수</strong>의 설정
1. 매개변수를 활용한 <strong>모델의 정의</strong>
1. 모델을 평가할 <strong>손실함수</strong>를 정의

In [3]:
# 2. 신경망 모델 구성
# (1) 모델 매개변수 설정
tf.reset_default_graph()                                 # tf 매개변수를 초기화 
X = tf.placeholder(tf.float32, [None, n_step, n_input])  # X 파라미터 3개 입력가능
Y = tf.placeholder(tf.float32, [None, n_class])          

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))
# BasicRNNCell, BasicLSTMCell, GRUCell : 이 함수들을 사용하면, 구조변경에 용이하다
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

In [4]:
# (2) 모델의 설정
# states = tf.zeros(batch_size)
# for i in range(n_step):
#     outputs, states = cell(X[[:, i]], states) ...원래는 이와 같이 복잡 했었다
# tf.nn.dynamic_rnn() : 레이어 함수로써 CNN 의 tf.nn.conv2d 함수처럼 간단하게 RNN 신경망을 생성한다
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [5]:
# 결과를 Y의 형식과 동일하게 맞춘다 cf) Y : [batch_size, n_class]
# ==> outputs : [batch_size, n_step, n_hidden]
#            -> [n_step, batch_size, n_hidden]
#            -> [batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

# (3) 손실함수의 정의
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### <strong>03 RNN 모델의 Train
tf.Session()

In [6]:
# 3. 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)

In [7]:
for epoch in range(total_epoch):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # X를 RNN 입력에 맞게 [batch_size, n_step, n_input] 형태로 변환
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
    if epoch % 3 == 0 :
        print('Epoch:{:4d}  Avg. cost = {:.3f}'.format( (epoch + 1), (total_cost / total_batch)))
print('최적화 완료!')

Epoch:   1  Avg. cost = 0.531
Epoch:   4  Avg. cost = 0.148
Epoch:   7  Avg. cost = 0.115
Epoch:  10  Avg. cost = 0.098
Epoch:  13  Avg. cost = 0.096
Epoch:  16  Avg. cost = 0.080
Epoch:  19  Avg. cost = 0.074
Epoch:  22  Avg. cost = 0.068
Epoch:  25  Avg. cost = 0.064
Epoch:  28  Avg. cost = 0.070
최적화 완료!


### <strong>04 Train 학습 모델의 평가
Accuracy()

In [8]:
# 4. 결과 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels
print('정확도:', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

정확도: 0.971


<br>
## <strong>2 단어 자동완성 
1. 자연어 처리나 음성 처리 분야에 많이 사용
1. 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성

<img src="https://www.kdnuggets.com/wp-content/uploads/RNN.jpg" align="left" width='400'>

### <strong>01 데이터 입력 및 모델의 파라미터 정의
Data

In [9]:
import tensorflow as tf
import numpy as np
# 예측할 알파벳 목록을 입력한다
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# one-hot 인코딩을 위한 연관 배열의 생성 : {'a':0, 'b':1, 'c':2, ... ,}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [10]:
# input_batch 와 target_batch 는 알파벳 배열의 인덱스 번호로 변환
# [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
def make_batch(seq_data):
    input_batch, target_batch = [], []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]  # 3, 3, 15, 4, 3 ...
        target = num_dic[seq[-1]]               # one-hot 인코딩 변환
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
    return input_batch, target_batch

### <strong>02 RNN 모델 및 손실함수의 정의
1. 모델에 사용할 <strong>매개변수</strong>의 설정
1. 매개변수를 활용한 <strong>모델의 정의</strong>
1. 모델을 평가할 <strong>손실함수</strong>를 정의

In [11]:
# 1. RNN 신경망 모델 정의
# (1) 파라미터 설정 :  RNN 시퀀스 갯수  ex) 타입 스텝: [1 2 3] => 3
tf.reset_default_graph()      # tf 매개변수를 초기화 
learning_rate = 0.01
n_hidden, total_epoch, n_step = 128, 30, 3  
n_input = n_class = dic_len   # 출력, 입력값 모두 26개 알파벳으로 분류

In [12]:
# (2) 알파벳 one-hot 인코딩으로 26개 값으로 출력
# cost함수 : sparse_softmax_cross_entropy_with_logits 는 인덱스를 그대로 사용
# if) one-hot 인코딩 사용 시 : ( tf.float23, [None, n_class] )
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32,   [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class])) #, reuse=True)
b = tf.Variable(tf.random_normal([n_class]))           #, reuse=True)  #https://github.com/tensorflow/tensorflow/issues/799

In [13]:
# (3) RNN 셀을 생성
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)    # Dropout의 설정
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)         # 셀의 조합을 위한 셀을 추가로 생성
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2]) # 여러 셀을 조합한 RNN 셀을 생성
# tf.nn.dynamic_rnn 를 이용해 순환 신경망을 생성한다 (time_major = True)
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [14]:
# (4) 최종 결과는 one-hot 인코딩 형식으로 변환
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b
# sparse_softmax_cross_entropy_with_logits : index 를 바로 받는다
# cf) softmax_cross_entropy_with_logits : one-hot 을 받는다
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### <strong>03 RNN 모델의 Train
tf.Session()

In [15]:
# 2. RNN 신경망 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})
    if epoch % 3 == 0:
        print('Epoch: {:.4f} cost = {:.6f}'.format(epoch + 1, loss))
print('최적화 완료!')

Epoch: 1.0000 cost = 3.409979
Epoch: 4.0000 cost = 1.329201
Epoch: 7.0000 cost = 0.587946
Epoch: 10.0000 cost = 0.474487
Epoch: 13.0000 cost = 0.266031
Epoch: 16.0000 cost = 0.125695
Epoch: 19.0000 cost = 0.112749
Epoch: 22.0000 cost = 0.042894
Epoch: 25.0000 cost = 0.037758
Epoch: 28.0000 cost = 0.029880
최적화 완료!


### <strong>04 Train 학습 모델의 평가
Accuracy()

In [16]:
# 3. 모델의 성능을 평가 (레이블이 정수이므로 예측값도 정수로 변경)
prediction       = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)       # 입력값을 그대로 비교한다
accuracy         = tf.reduce_mean(tf.cast(prediction_check, tf.float32))
input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val     = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

In [17]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)
sess.close()


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'coll', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 0.9


<br>
## <strong>3 Sequence to Sequence
Seq2Seq는 구글 기계번역에 사용하는 신경망 모델이다 (http://aidev.co.kr/deeplearning/2273)
1. <strong>'RNN 인코더 셀'</strong> (A, B, C) 과 <strong>'RNN 디코더 셀'</strong> (W, X, Y, X) RNN 2개 모델을 결합한다
1. 기본모델에 <strong>특수함 심벌</strong> ((1) 입력의 시작을 알림 (2) 디코더 출력이 끝남을 알림)을 포함한다 
1. <strong>'챗봇', '번역', '이미지 캡셔닝'</strong> 등의 모델을 생성한다

<img src="https://isaacchanghau.github.io/images/nlp/seq2seq-neuralconver/model-example.png" align="left" width='600'>

### <strong>01 데이터 입력 및 모델의 파라미터 정의
Data

In [18]:
# 영어 단어를 한국어 단어로 번역하는 프로그램을 만들어봅니다.
# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd'] ||  to   -> ['t', 'o', 'P', 'P']
import tensorflow as tf
import numpy as np

# 1. Input Data를 설정한다
# (1) elements 를 정의한 뒤 (알파벳과 한글), {dict} 데이터 형식으로 변환
char_arr = [ c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑'] 
num_dic  = { n : i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

In [19]:
# (2) 번역학습 데이터 입력
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], 
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

# (3) 학습데이터를 '심볼'을 포함한 One-Hot-Encoding 으로 변환
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]           # 인코더 셀 입력값 : 입력단어를 한글자씩 떼어 배열생성
        output = [num_dic[n] for n in ('S' + seq[1])]  # 디코더 셀 입력값 : 시작을 나타내는 'S'심볼을 맨 앞에 붙인다
        target = [num_dic[n] for n in (seq[1] + 'E')]  # 디코더 셀 출력값 : 비교값으로, 끝을 알리기 위해 끝에 'E'를 붙인다
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아니다 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)
    return input_batch, output_batch, target_batch

### <strong>02 RNN 모델 및 손실함수의 정의
1. 모델에 사용할 <strong>매개변수</strong>의 설정
1. 매개변수를 활용한 <strong>모델의 정의</strong>
1. 모델을 평가할 <strong>손실함수</strong>를 정의

In [20]:
# 2. 모델의 정의
# (1) 파라미터값 설정
tf.reset_default_graph()      # tf 매개변수를 초기화 
learning_rate = 0.01
n_hidden, total_epoch = 128, 100
n_class = n_input = dic_len  # 입력과 출력 : one-hot 인코딩으로 크기가 같다

# (2) 인코더/ 디코더 입 출력 설정
# Seq2Seq은 '인코더 입력'과 '디코더 입력'의 형식이 같다 : [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])  # [batch size, time steps]

In [21]:
# (3) 인코더/ 디코더 셀 설정
# 1) tf.variable_scope() : 인코더 셀을 정의한다
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob = 0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype = tf.float32)
# 2) 디코더 셀을 정의한다
# '인코더 셀'의 '최종 상태값'을, '디코더 셀'의 '초기 상태값'과 연결한다
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                                            initial_state = enc_states, 
                                            dtype = tf.float32)

In [22]:
# (4) 설정한 매개변수와 Cell을 연결하여 Seq2Seq 모델을 생성한다
model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits = model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### <strong>03 RNN 모델의 Train
tf.Session()

In [23]:
# 3. 모델의 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 5 == 0 :
        print('Epoch: {:4d}  cost = {:.6f}'.format((epoch + 1),loss))
print('최적화 완료!')

Epoch:    1  cost = 3.838162
Epoch:    6  cost = 0.368465
Epoch:   11  cost = 0.102032
Epoch:   16  cost = 0.120568
Epoch:   21  cost = 0.059999
Epoch:   26  cost = 0.090998
Epoch:   31  cost = 0.018229
Epoch:   36  cost = 0.001458
Epoch:   41  cost = 0.001873
Epoch:   46  cost = 0.002172
Epoch:   51  cost = 0.002402
Epoch:   56  cost = 0.000906
Epoch:   61  cost = 0.002370
Epoch:   66  cost = 0.000550
Epoch:   71  cost = 0.001295
Epoch:   76  cost = 0.000572
Epoch:   81  cost = 0.000494
Epoch:   86  cost = 0.000359
Epoch:   91  cost = 0.000312
Epoch:   96  cost = 0.000439
최적화 완료!


### <strong>04 Train 학습 모델의 평가
Accuracy()

In [24]:
# 4. 모델의 평가 (번역 테스트) : 입력받은 단어의 번역을 예측하고 디코딩하는 함수
# (1) 예측시 사전에 없는 단어는 'P'로 채운다. ex) ['word', 'PPPP']
# (2) [batch size, time step, input] 를 예측결과로 출력 : 2번째 차원값 input차원의 argmax로 예측한다
def translate(word):
    seq_data = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    decoded = [char_arr[i] for i in result[0]]  # 결과 값 '인덱스'에 해당글자를 사용하여 배열을 만든다.
    end = decoded.index('E')                    # 출력끝 을 의미하는 'E' 이후의 글자들은 제거한다
    translated = ''.join(decoded[:end])
    return translated

print('\n=== 번역 테스트 ===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 키스
